## Imports

In [1]:
import os
import glob
from natsort import natsorted

## Define Parameters

In [2]:
SURGERY_FOLDER = '11' # This number has to be 11, since it is the number we chose when preprocessing the test data
DATA_ROOT = '/mnt/sds-hd/sd22a004/guest/data_preprocessed/data_testset'
SPLIT_PATH = '/mnt/sds-hd/sd22a004/guest/data_preprocessed/splits/data_splitted_testset'

## Defining functions

In [3]:
def get_images_in_surgery_folder(data_root, surgery_folder):
    """
    In a specific surgery dir like '2019_xx_xx_aa1111', it returns all the image_paths as a list
    :param surgery_folder:
    :return: indices, surgery_names
    The surgery names are '2019_xx_xxxx' will be used to group the data so that all the images belonging
    to one op remain in the same split created by sklearn
    """
    indices, surgery_names = [], []
    image_filepaths=[]
    image_filepaths_l = natsorted(glob.glob(os.path.join(data_root, surgery_folder,'image_02','images', '*')))
    image_filepaths_r = natsorted(glob.glob(os.path.join(data_root, surgery_folder,'image_03','images', '*')))
    for a, b in zip(image_filepaths_l, image_filepaths_r):
        image_filepaths.append(a)
        image_filepaths.append(b)
    # Prepare text file information
    rel_path_name = os.path.join(surgery_folder)  # Folder name
    frame_indices = [os.path.basename(os.path.splitext(path)[0]) for path in image_filepaths]
    newline_batch = [' '.join((rel_path_name, frame_index)) for frame_index in frame_indices]
    indices += newline_batch
    # Just append surgery name along as the 'group' variable
    surgery_names += [surgery_folder] * len(image_filepaths)
   
    return indices, surgery_names



def append_lr_to_filenames(indices):
    appended_indices = []
    for i, line in enumerate(indices):
        if i % 2 == 0:
            index = ' '.join((line, 'l'))
        else:
            index = ' '.join((line, 'r'))
        appended_indices.append(index)
    return appended_indices

## Create list

In [4]:
indices, surgery_names = get_images_in_surgery_folder(DATA_ROOT, SURGERY_FOLDER)
filenames = append_lr_to_filenames(indices)

Create necessary directory structure. This will throw an error if the directory already exists, to prevent overwriting files.  
If overwriting is desired set the `EXIST_OK` flag to `True`.

In [5]:
# Create necessary directory structure. This will throw an error if the directory already exists, to prevent overwriting files.
# If overwriting is desired set the EXIST_OK flag to True.
EXIST_OK = False
try:
    os.makedirs
    os.makedirs(os.path.join(SPLIT_PATH, 'test_cv_split_lr/'))
    with open(os.path.join(SPLIT_PATH, 'test_cv_split_lr/test_file.txt'),'w') as fp:
        for item in filenames:
            fp.write("%s\n" % item)
        print('Done')
except FileExistsError:
    print('The directory already exists. To allow overwriting existing files set the EXIST_OK flag to True.')

Done
